In [1]:
import joblib
import os
from API.utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
os.listdir('./model')

['ANN(58_to_sp162).pkl',
 'SVR(5_to_23).pkl',
 'transformer(23_to_54).pkl',
 'transformer(SP162_to_Y162).pkl']

In [2]:
os.listdir('./data/phase_2/cleaned')

['col_names.pkl', 'train.csv']

# load pretrained model

In [3]:
A = joblib.load('./model/SVR(5_to_23).pkl')
B = joblib.load('./model/transformer(23_to_54).pkl')
C = joblib.load('./model/ANN(58_to_sp162).pkl')
D = joblib.load('./model/transformer(SP162_to_Y162).pkl')
col_names = joblib.load('./data/phase_2/cleaned/col_names.pkl')

# step1

In [4]:
df = pd.read_csv('./data/phase_1/test_170.csv',index_col=0).sample(1)
y_test = df.iloc[:,4:]
X_test = df[['T10','T50','T90','N+A']]
X_test

,T10,T50,T90,N+A
140,99.8,120.3,146.6,41.35


In [5]:
y_pred23 = A.predict(X_test)
y_pred23 = pd.DataFrame(y_pred23,columns=y_test.columns)
print(y_pred23.sum(axis=1))
t = y_pred23.append(y_test)
t.index = ['pred','real']
t.loc['error'] = abs(t.T['pred'] - t.T['real'].T)
t['sum'] = t.sum(axis=1)
t

0    100.0
dtype: float64


,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,sum
pred,1.089159,0.749873,0.155162,3.313611,2.114835,3.516061,0.671724,7.62816,5.990579,9.764872,3.343085,8.011297,8.335899,7.363796,6.229498,5.251753,9.574435,5.079525,3.778642,1.206075,5.384325,0.687115,0.760521,100.00000
real,1.018000,0.664000,0.126000,3.680000,2.234000,3.445000,0.563000,7.93200,6.181000,9.106000,3.034000,7.836000,8.159000,6.266000,9.132000,5.118000,9.093000,4.336000,4.052000,1.058000,5.102000,0.548000,0.742000,99.42500
error,0.071159,0.085873,0.029162,0.366389,0.119165,0.071061,0.108724,0.30384,0.190421,0.658872,0.309085,0.175297,0.176899,1.097796,2.902502,0.133753,0.481435,0.743525,0.273358,0.148075,0.282325,0.139115,0.018521,8.88635


# step 2

In [6]:
y_pred54 = B(y_pred23)
print(y_pred54.sum(axis=1))
y_pred54.head()

0    100.0
dtype: float64


,O2,METHANE,ETHANE,PROPANE,BUTANE,IPENTANE,PENTANE,T2PENTEN,CP,3MP,T2HEXENE,HEXANE,MCP,22MP,BENZENE,CH,223MB,4M1HEXEN,2MHX,1T3M,HEPTANE,H2O,MCH,ECP,23DM1HX,TOLUENE,2MHEPT,OCTANE,ECH,EBENZENE,PXYLENE,MXYLENE,OXYLENE,2MOCTN,1NONENE,NONANE,CUMENE,1T35TMCH,PCH,PRBENZEN,135MBENZ,1M2EBNZN,123MBENZ,24DMOCT,TEMH,3MN,TBCYHXAN,DECANE,ODEBZ,UNDECANE,PNBZ,DODECANE,HXBNZN,TRIDECAN
0,0.0,0.0,0.0,0.037494,0.074987,0.622395,1.089159,0.014997,0.155162,2.072538,0.042297,3.313611,2.281068,0.573818,0.671724,1.234993,0.630507,0.012577,4.773677,0.90389,7.62816,0.0,5.235918,3.625063,0.083359,3.343085,8.25254,8.011297,7.363796,1.479225,0.83911,2.071136,1.840027,9.331219,0.243216,5.251753,0.053556,3.047576,2.031949,0.228438,0.98102,1.449836,1.065792,0.905447,1.343458,3.13542,0.687115,1.01138,0.711163,0.191284,0.041346,0.00341,0.008013,0.0


# step3

In [7]:
df = pd.read_csv('./data/phase_2/cleaned/train.csv',index_col=0).head(1)
x = df[col_names['case']+col_names['xna']] #58
sp = df[col_names['sle']+col_names['shc']+col_names['she']] #162
y = df[col_names['xle']+col_names['xhc']+col_names['xhe']] #162
df

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C7+ in Light End (Input)_wt%,Case Conditions_C6P- in Heart Cut (Input)_wt%,Naphtha Properties_Oxygen_wt%,Naphtha Properties_Methane_wt%,Naphtha Properties_Ethane_wt%,Naphtha Properties_Propane_wt%,Naphtha Properties_n-Butane_wt%,Naphtha Properties_i-Pentane_wt%,Naphtha Properties_n-Pentane_wt%,Naphtha Properties_tr2-Pentene_wt%,Naphtha Properties_Cyclopentane_wt%,Naphtha Properties_3-Methylpentane_wt%,Naphtha Properties_tr2-Hexene_wt%,Naphtha Properties_n-Hexane_wt%,Naphtha Properties_Methylcyclopentane_wt%,"Naphtha Properties_2,2-Dimethylpentane_wt%",Naphtha Properties_Benzene_wt%,Naphtha Properties_Cyclohexane_wt%,"Naphtha Properties_2,2,3-Trimethylbutane_wt%",Naphtha Properties_4-Methyl-1-hexene_wt%,Naphtha Properties_2-Methylhexane_wt%,"Naphtha Properties_t-1,3-Dimethylcyclopentane_wt%",Naphtha Properties_n-Heptane_wt%,Naphtha Properties_H2O_wt%,Naphtha Properties_Methylcyclohexane_wt%,Naphtha Properties_Ethylcyclopentane_wt%,"Naphtha Properties_2,3-Dimethyl-1-hexene_wt%",Naphtha Properties_Toluene_wt%,Naphtha Properties_2-Methylheptane_wt%,Naphtha Properties_n-Octane_wt%,Naphtha Properties_Ethylcyclohexane_wt%,Naphtha Properties_EthylBenzene_wt%,Naphtha Properties_P-Xylene_wt%,Naphtha Properties_M-Xylene_wt%,Naphtha Properties_O-Xylene_wt%,Naphtha Properties_2-Methyloctane_wt%,Naphtha Properties_1-Nonene_wt%,Naphtha Properties_n-Nonane_wt%,Naphtha Properties_i-Propylbenzene_wt%,"Naphtha Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Naphtha Properties_n-Propylcyclohexane_wt%,Naphtha Properties_n-Propylbenzene_wt%,"Naphtha Properties_1,3,5-Trimethylbenzene_wt%",Naphtha Properties_1-Methyl-2-ethylbenzene_wt%,"Naphtha Properties_1,2,3-Trimethylbenzene_wt%","Naphtha Properties_2,4-Dimethyloctane_wt%","Naphtha Properties_2,2,3,3-Tetramethylhexane_wt%",Naphtha Properties_3-Methylnonane_wt%,Naphtha Properties_tert-Butylcyclohexane_wt%,Naphtha Properties_n-Decane_wt%,"Naphtha Properties_1,2-Diethylbenzene_wt%",Naphtha Properties_n-Undecane_wt%,Naphtha Properties_n-Pentylbenzene_wt%,Naphtha Properties_n-Dodecane_wt%,Naphtha Properties_n-Hexylbenzene_wt%,Naphtha Properties_n-Tridecane_wt%,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexa

In [8]:
sp_pred = C.predict(x)[sp.columns.tolist()]
print(sp_pred.sum(axis=1))
t = sp_pred.append(sp)
t.index = ['pred','real']
t.loc['error'] = abs(t.T['pred'] - t.T['real'])
t['sum'] = t.sum(axis=1)
t

0    54.0
dtype: float32


,Individual Component to Light End Split Factor_Oxygen_Ratio,Individual Component to Light End Split Factor_Methane_Ratio,Individual Component to Light End Split Factor_Ethane_Ratio,Individual Component to Light End Split Factor_Propane_Ratio,Individual Component to Light End Split Factor_n-Butane_Ratio,Individual Component to Light End Split Factor_i-Pentane_Ratio,Individual Component to Light End Split Factor_n-Pentane_Ratio,Individual Component to Light End Split Factor_tr2-Pentene_Ratio,Individual Component to Light End Split Factor_Cyclopentane_Ratio,Individual Component to Light End Split Factor_3-Methylpentane_Ratio,Individual Component to Light End Split Factor_tr2-Hexene_Ratio,Individual Component to Light End Split Factor_n-Hexane_Ratio,Individual Component to Light End Split Factor_Methylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,2-Dimethylpentane_Ratio",Individual Component to Light End Split Factor_Benzene_Ratio,Individual Component to Light End Split Factor_Cyclohexane_Ratio,"Individual Component to Light End Split Factor_2,2,3-Trimethylbutane_Ratio",Individual Component to Light End Split Factor_4-Methyl-1-hexene_Ratio,Individual Component to Light End Split Factor_2-Methylhexane_Ratio,"Individual Component to Light End Split Factor_t-1,3-Dimethylcyclopentane_Ratio",Individual Component to Light End Split Factor_n-Heptane_Ratio,Individual Component to Light End Split Factor_H2O_Ratio,Individual Component to Light End Split Factor_Methylcyclohexane_Ratio,Individual Component to Light End Split Factor_Ethylcyclopentane_Ratio,"Individual Component to Light End Split Factor_2,3-Dimethyl-1-hexene_Ratio",Individual Component to Light End Split Factor_Toluene_Ratio,Individual Component to Light End Split Factor_2-Methylheptane_Ratio,Individual Component to Light End Split Factor_n-Octane_Ratio,Individual Component to Light End Split Factor_Ethylcyclohexane_Ratio,Individual Component to Light End Split Factor_EthylBenzene_Ratio,Individual Component to Light End Split Factor_P-Xylene_Ratio,Individual Component to Light End Split Factor_M-Xylene_Ratio,Individual Component to Light End Split Factor_O-Xylene_Ratio,Individual Component to Light End Split Factor_2-Methyloctane_Ratio,Individual Component to Light End Split Factor_1-Nonene_Ratio,Individual Component to Light End Split Factor_n-Nonane_Ratio,Individual Component to Light End Split Factor_i-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1-trans-3,5-Trimethylcyclohexane_Ratio",Individual Component to Light End Split Factor_n-Propylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Propylbenzene_Ratio,"Individual Component to Light End Split Factor_1,3,5-Trimethylbenzene_Ratio",Individual Component to Light End Split Factor_1-Methyl-2-ethylbenzene_Ratio,"Individual Component to Light End Split Factor_1,2,3-Trimethylbenzene_Ratio","Individual Component to Light End Split Factor_2,4-Dimethyloctane_Ratio","Individual Component to Light End Split Factor_2,2,3,3-Tetramethylhexane_Ratio",Individual Component to Light End Split Factor_3-Methylnonane_Ratio,Individual Component to Light End Split Factor_tert-Butylcyclohexane_Ratio,Individual Component to Light End Split Factor_n-Decane_Ratio,"Individual Component to Light End Split Factor_1,2-Diethylbenzene_Ratio",Individual Component to Light End Split Factor_n-Undecane_Ratio,Individual Component to Light End Split Factor_n-Pentylbenzene_Ratio,Individual Component to Light End Split Factor_n-Dodecane_Ratio,Individual Component to Light End Split Factor_n-Hexylbenzene_Ratio,Individual Component to Light End Split Factor_n-Tridecane_Ratio,Individual Component to Heart Cut Split Factor_Oxygen_Ratio,Individual Component to Heart Cut Split Factor_Methane_Ratio,Individual Component to Heart Cut Split Factor_Ethane_Ratio,Individual Component to Heart Cut Split Factor_Propane_Ratio,Individual Component to Heart Cut Split Factor_n-Butane_Ratio,Individual Component to Hea

# step4

In [9]:
xna = df[col_names['xna']]
y_pred = D(xna,sp_pred)
print(y_pred.sum(axis=1))
t = y_pred.append(y)
t.index = ['pred','real']
t.loc['error'] = abs(t.T['pred'] - t.T['real'])
t['sum'] = t.sum(axis=1)
t

0    300.0
dtype: float64


,Light End Product Properties_Oxygen_wt%,Light End Product Properties_Methane_wt%,Light End Product Properties_Ethane_wt%,Light End Product Properties_Propane_wt%,Light End Product Properties_n-Butane_wt%,Light End Product Properties_i-Pentane_wt%,Light End Product Properties_n-Pentane_wt%,Light End Product Properties_tr2-Pentene_wt%,Light End Product Properties_Cyclopentane_wt%,Light End Product Properties_3-Methylpentane_wt%,Light End Product Properties_tr2-Hexene_wt%,Light End Product Properties_n-Hexane_wt%,Light End Product Properties_Methylcyclopentane_wt%,"Light End Product Properties_2,2-Dimethylpentane_wt%",Light End Product Properties_Benzene_wt%,Light End Product Properties_Cyclohexane_wt%,"Light End Product Properties_2,2,3-Trimethylbutane_wt%",Light End Product Properties_4-Methyl-1-hexene_wt%,Light End Product Properties_2-Methylhexane_wt%,"Light End Product Properties_t-1,3-Dimethylcyclopentane_wt%",Light End Product Properties_n-Heptane_wt%,Light End Product Properties_H2O_wt%,Light End Product Properties_Methylcyclohexane_wt%,Light End Product Properties_Ethylcyclopentane_wt%,"Light End Product Properties_2,3-Dimethyl-1-hexene_wt%",Light End Product Properties_Toluene_wt%,Light End Product Properties_2-Methylheptane_wt%,Light End Product Properties_n-Octane_wt%,Light End Product Properties_Ethylcyclohexane_wt%,Light End Product Properties_EthylBenzene_wt%,Light End Product Properties_P-Xylene_wt%,Light End Product Properties_M-Xylene_wt%,Light End Product Properties_O-Xylene_wt%,Light End Product Properties_2-Methyloctane_wt%,Light End Product Properties_1-Nonene_wt%,Light End Product Properties_n-Nonane_wt%,Light End Product Properties_i-Propylbenzene_wt%,"Light End Product Properties_1-trans-3,5-Trimethylcyclohexane_wt%",Light End Product Properties_n-Propylcyclohexane_wt%,Light End Product Properties_n-Propylbenzene_wt%,"Light End Product Properties_1,3,5-Trimethylbenzene_wt%",Light End Product Properties_1-Methyl-2-ethylbenzene_wt%,"Light End Product Properties_1,2,3-Trimethylbenzene_wt%","Light End Product Properties_2,4-Dimethyloctane_wt%","Light End Product Properties_2,2,3,3-Tetramethylhexane_wt%",Light End Product Properties_3-Methylnonane_wt%,Light End Product Properties_tert-Butylcyclohexane_wt%,Light End Product Properties_n-Decane_wt%,"Light End Product Properties_1,2-Diethylbenzene_wt%",Light End Product Properties_n-Undecane_wt%,Light End Product Properties_n-Pentylbenzene_wt%,Light End Product Properties_n-Dodecane_wt%,Light End Product Properties_n-Hexylbenzene_wt%,Light End Product Properties_n-Tridecane_wt%,Heart Cut Product Properties_Oxygen_wt%,Heart Cut Product Properties_Methane_wt%,Heart Cut Product Properties_Ethane_wt%,Heart Cut Product Properties_Propane_wt%,Heart Cut Product Properties_n-Butane_wt%,Heart Cut Product Properties_i-Pentane_wt%,Heart Cut Product Properties_n-Pentane_wt%,Heart Cut Product Properties_tr2-Pentene_wt%,Heart Cut Product Properties_Cyclopentane_wt%,Heart Cut Product Properties_3-Methylpentane_wt%,Heart Cut Product Properties_tr2-Hexene_wt%,Heart Cut Product Properties_n-Hexane_wt%,Heart Cut Product Properties_Methylcyclopentane_wt%,"Heart Cut Product Properties_2,2-Dimethylpentane_wt%",Heart Cut Product Properties_Benzene_wt%,Heart Cut Product Properties_Cyclohexane_wt%,"Heart Cut Product Properties_2,2,3-Trimethylbutane_wt%",Heart Cut Product Properties_4-Methyl-1-hexene_wt%,Heart Cut Product Properties_2-Methylhexane_wt%,"Heart Cut Product Properties_t-1,3-Dimethylcyclopentane_wt%",Heart Cut Product Properties_n-Heptane_wt%,Heart Cut Product Properties_H2O_wt%,Heart Cut Product Properties_Methylcyclohexane_wt%,Heart Cut Product Properties_Ethylcyclopentane_wt%,"Heart Cut Product Properties_2,3-Dimethyl-1-hexene_wt%",Heart Cut Product Properties_Toluene_wt%,Heart Cut Product Properties_2-Methylheptane_wt%,Heart Cut Product Properties_n-Octane_wt%,Heart Cut Product Properties_Ethylcyclohexane_wt%,Heart Cut Product Properties_EthylBenzene_wt%,Heart Cut Product

In [10]:
print(y_pred[col_names['xle']].sum(axis=1))
print(y_pred[col_names['xhc']].sum(axis=1))
print(y_pred[col_names['xhe']].sum(axis=1))

0    100.0
dtype: float64
0    100.0
dtype: float64
0    100.0
dtype: float64
